# Musing on implementing semigroup representation theory and software integration
$\newcommand{\K}{\mathbb{K}}$
<img src="pictures/banner.png" align="right" width="55%" style="opacity:0.5;filter:alpha(opacity=40);"/>

Nicolas M. Thiéry<br>
LRI, Université Paris Sud

View the live slides on binder: https://tinyurl.com/swavl3h
<a href="https://mybinder.org/v2/gh/OpenDreamKit/demo-semigroup-representation-theory/master?filepath=2020-01-18-INI-implementing-semigroup-representation-theory.ipynb">
<img src="pictures/2020-01-18-INI-implementing-semigroup-representation-theory.qrcode.png" width="25%"></a>

<img src="pictures/Flag_of_Europe.svg"   width="10%" align="left">
<img src="pictures/odk-elected-logo.svg" width="6%" align="right">


## Abstract

Extending representation theory from finite groups to finite
semigroups brings interesting challenges, combinatorics, and
applications. Almost a decade ago, I proposed an algorithm to compute
the Cartan Matrix of a semigroup algebra -- a combinatorial invariant
that contains information on how projective modules are built from
simple modules. It boils down to computing with finite semigroups,
characters of groups, and combinatorics. Despite this relative
simplicity, and much to my embarrassment, a full production-grade
implementation is only finally in reach.

In this talk, I will report on ongoing joint work with my PhD student
Balthazar Charles to implement this algorithm and adapt it to modular representations,
and use this occasion to illustrate the evolution of our computational
landscape toward an ecosystem of interoperable software, thanks to large
scale collaborations.

## Red line of the talk

Balthazar is starting a PhD in <span class="modular">modular</span> representation theory of <span class="semigroup">semigroups</span>

### Mathematical landscape

- Representation theory of groups: a grand old topic; 10k+ papers in a century 
- Representation theory of the <span class=sym>symmetric group $\mathfrak{S}_n$</span>: beautiful combinatorics

- <span class=modular>Modular</span> representation theory of groups: 1k+ papers in the past 50 years

- Representation theory of <span class=semigroup>semigroups</span>: dozens of papers in the last decades

- Motto of <span class=semigroup>semigroup theory</span>: reduce to **group theory** and **combinatorics**

### Strategy

- Implement an algorithm of 2010 in full generality<br>
  <span class=semigroup>Computing the Cartan invariants matrix of a semigroup</span>
- Adapt this algorithm to the <span class=modular>modular</modular> case
- Explore the representation theory of the <span class=sym>transformation semigroup $T_n$</span>: beautiful combinatorics?

## The mathematical landscape

### Semigroup theory perspective
- Groups as perfect circles

- The imperfection of semigroups: what happens when you can't always run in circle?

- Irreversible steps, **ideals** (left and right)

- Invertibility lost. All hopes for structure lost?

- May the Associativity be with you:  **J-classes**, **eggbox picture**

### Why bother?

- Representation theory: also a tool for studying processes<br>
  Example: Splitting the state space of a large discrete Markov chain

- Life is imperfect: many processes have irreversible steps!

**Example:** the Tsetlin library

<img src="pictures/tsetlin-library.png" width=40% align="right">

- Beautiful eigenvalues!
- Explanation: very irreversible process<br> $\Longrightarrow$ basic representation theory

### There are other angels

<img src="pictures/monoidClasses.svg" align="center" width="50%">

**Motto**: reduce to the angels: group theory, combinatorics, linear algebra

### Representation theory of semigroups, in a nutshell

- Not semi-simple: simple ≠ indecomposable

- **Composition factors**

- Construction of simple modules: induced from the simple modules of the groupes

- Character theory: trace of the action of conjugacy class representatives<br>
  $\Longrightarrow$ tool to recover composition factors

- **Projective indecomposable modules**: “the largest indecomposable modules”

- **The Cartan invariants matrix**: composition factors of projective indecomposable modules<br>
  $\Longrightarrow$ a nice source of combinatorics

### Computing the Cartan Invariant matrix?

**Algorithm from finite dimensional algebras:**
- Decompose the center of the semisimple quotient (MeatAxe-style)
- Recover idempotents
- Compute projective modules
- Recover composition factors

$\Longrightarrow$ Linear algebra on $\K S$, if not $End(\K S)$

Intractable for $\#S = 31103$    (bihecke monoid of type $A_5$)

**Basic insight (T. 2010):**
- Realize that the Cartan invariant matrix is the character of $\K S$ as $S$-mod-$S$ bimodule
- Computation by characters!

**Algorithm:**
- Compute semigroup structure, conjugacy class representatives $CC$
- Compute the character table of $S$ from character table of $G$<br>
  Probable bottleneck: computing the radical of $\K R$: $O\left(\sum\#R_J^3\right)$
- Count two sided fixed points for each pair in $CC$ acting on $S$<br>
  Complexity: $O\left(\#CC^2\sum_J n_J+ m_J\right)$ group operations (Charles, Mitchell, T. '20)
- Recover the composition factors

**Pros:**
- Embarassingly parallel!
- Amenable to <span class="modular">modular representations</span>: use Brauer characters

**Implementation:**
- Good: handles my semigroup of size 31k in one hour
- Bad: aperiodic case only so far!

## Musing on our computational landscape

### Available building blocks for semigroup representation theory

- Group computations: GAP, Magma
- Semigroup computations: GAP, Semigroups (GAP), KBMag (C), Semigroupe (C), libsemigroups (C++), Sage
- (Modular) representation theory of groups: GAP (but also GAP3+Specht)
- Algebraic Combinatorics: Sage, Symmetrica (C), lrcalc (C), ...
- Linear algebra: MeatAxe, Linbox, Sage, ...
- Coxeter groups: GAP3+Chevie, GAP, Sage, Coxeter3, ...
- Markov chains: Mathematica, ...

Question: How to proceed?

### The Unicorn way: “There can be only one”

That's what happens in the tech industry: a single player takes it all (Amazon, AirBnB, Uber, ...) 

Why not for us?

Let's reimplement everything in ~~C++~~, ~~Magma~~, ~~GAP~~, ~~Sage~~, ~~Julia~~, ~~Mathematica~~ your favorite system

- Ok for a quick focused result<br>
  Maybe that's what I should have done, actually, to get my paper out

- Does not scale, due to time scale and man power:<br>
  Our software are the result of decades of hard work and deep expertise

- Promotes competition between systems, at the wrong scale

- Honestly, we don't know what will a good system in 10-20 years<br>
  we need to explore many approaches

### Balthazar's tool box

In [1]:
import sage_annotations
from mygap import mygap
mygap.LoadPackage("Semigroups");

import sage_semigroups
import sage_combinat_widgets

from   sage_explorer import explore
from sage_explorer.sage_explorer import Settings
Settings.add_property('cardinality', predicate=Groups().Finite().__contains__)
Settings.add_property('conjugacy_classes', predicate=Groups().Finite().__contains__)
Settings.add_property('multiplication_table', predicate=Groups().Finite().__contains__)
#Partitions.options.convention="French"
%display unicode_art
tensor.symbol = " ⊗ "
%run style/odk.py

#### <span class=semigroup>Semigroup theory</a>

In [2]:
T3 = mygap.FullTransformationSemigroup(3)

In [3]:
graph = T3.cayley_graph()
graph.set_latex_options(format="dot2tex")
view(graph)

In [4]:
from francy_widget import FrancyWidget
from networkx import DiGraph
g = DiGraph()
g.add_edges_from([(e[0], e[1]) for e in graph.edges()])
FrancyWidget(g)

FrancyWidget(value=<networkx.classes.digraph.DiGraph object at 0x7f44bb028898>)

In [5]:
T5 = mygap.FullTransformationSemigroup(5)

In [6]:
T5.cardinality()

3125

In [7]:
d_classes = T5.d_classes()
for d_class in d_classes:
    print(d_class)

<Green's D-class: IdentityTransformation>
<Green's D-class: Transformation( [ 1, 2, 3, 4, 1 ] )>
<Green's D-class: Transformation( [ 1, 1, 2, 3, 1 ] )>
<Green's D-class: Transformation( [ 3, 1, 3, 1, 3 ] )>
<Green's D-class: Transformation( [ 1, 1, 1, 1, 1 ] )>


In [8]:
G = d_classes[1].schutzenberger_group()
G

Group([ (1,2,3,4), (1,2) ])

In [9]:
G = d_classes[1].schutzenberger_group()
G

Group([ (1,2,3,4), (1,2) ])

#### <span class=modular>Modular representation of groups</span>

In [10]:
reps = G.irreducible_representations(GF(3))
for rho in reps:
    display([matrix(rho(g).gap()) for g in G.group_generators()])

[ (2), (2) ]

[ (1), (1) ]

⎡ ⎛2 2 0⎞  ⎛2 0 0⎞ ⎤
⎢ ⎜2 1 0⎟  ⎜0 2 2⎟ ⎥
⎣ ⎝0 1 1⎠, ⎝0 0 1⎠ ⎦

⎡ ⎛0 1 2⎞  ⎛2 2 0⎞ ⎤
⎢ ⎜2 2 1⎟  ⎜0 1 0⎟ ⎥
⎣ ⎝2 2 0⎠, ⎝0 0 1⎠ ⎦

In [11]:
all( [ rho(g)*rho(h) == rho(g*h) for g in G for h in G ] )

True

#### Sage-GAP lightweight Math-in-the-Middle interface

In [12]:
A = T5.algebra(QQ); A

Algebra of <full transformation monoid of degree 5> over Rational Field

In [13]:
A.an_element() ^ 3

58*B                                    + 72*
    Transformation( [ 4, 5, 1, 2, 3 ] )      

B                                    + 76*1 + 63*
 Transformation( [ 2, 3, 4, 5, 1 ] )             

B                                    + 74*B
 Transformation( [ 5, 1, 2, 3, 4 ] )       Transformation( [ 3, 4, 5, 1, 2 ] )

##### How it works
- Enriched libgap handles with
- Semantic carried over using
- Alignments provided as annotations
```python
    @semantic(gap="Group", variant="multiplicative")
    class Groups:

        class ParentMethods:

            @semantic(gap="GeneratorsOfGroup", codomain=Family[Self])
            @abstract_method
            def group_generators(self):
                pass
```

#### Exploring

In [14]:
explore(G)

SageExplorer(children=(VBox(children=(ExplorerTitle(children=(MathTitle(value='Exploring: Group([ (1,2,3,4), (…

#### <span class=sym>Combinatorial Representation Theory of $\mathfrak S_n$</sym>

In [15]:
StandardTableaux(10).random_element()

┌───┬────┬───┬───┐
│ 1 │ 4  │ 5 │ 9 │
├───┼────┼───┴───┘
│ 2 │ 7  │
├───┼────┤
│ 3 │ 10 │
├───┼────┘
│ 6 │
├───┤
│ 8 │
└───┘

In [16]:
Sym = SymmetricFunctions(QQ['t']);
s = Sym.s()

In [17]:
s[3,1].coproduct()

1 ⊗ s     + s   ⊗ s    + s   ⊗ s     + s   ⊗ s    + s    ⊗ s   + s    ⊗ s    + 
     ┌┬┬┐    ┌┐    ┌┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┐    ┌┬┐    ┌┐    ┌┬┐    ┌┬┐   
     ├┼┴┘    └┘    ├┼┘    └┘    └┴┴┘    ├┤    └┴┘    └┴┘    ├┤    └┴┘    └┴┘   
     └┘            └┘                   └┘                  └┘                 

s    ⊗ s   + s     ⊗ s   + s     ⊗ 1
 ┌┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┬┐   
 ├┼┘    └┘    └┴┴┘    └┘    ├┼┴┘   
 └┘                         └┘     

In [18]:
@interact
def f(p1 = Partition([2,1])._widget_()):
      return s[p1].coproduct()

Interactive function <function f at 0x7f44b8ddd730> with 1 widget
  p1: GridViewWidget(value=[2, 1], children=…

#### <span class=modular>Modular</span> combinatorial representation theory of <span class=sym>$\mathfrak S_n$</a>

In [19]:
list(RibbonTableaux([[5,4,3],[2,1]], [2,1], 3))

⎡   .  .  0  0  0    .  .  1  0  0    .  .  0  0  0 ⎤
⎢   .  0  0  2       .  0  0  0       .  1  0  1    ⎥
⎣   1  0  1      ,   1  0  2      ,   2  0  0       ⎦

In [20]:
Sym.llt(3)

level 3 LLT polynomials over Univariate Polynomial Ring in t over Rational Field

#### Reproducible environment

Complete description of the environment:

    FROM registry.gitlab.com/sagemath/sage/sagemath-dev:9.0-py3

    RUN sudo apt-get update && sudo apt-get -qy install graphviz build-essential git g++ && sudo apt-get clean
    RUN sage -i gap_packages && rm -rf /home/sage/sage/upstream
    RUN sudo apt-get update && sudo apt-get -qq install -y curl \
        &&  curl -sL https://deb.nodesource.com/setup_10.x | sudo -E bash - \
        && sudo apt-get install -yq nodejs && sudo npm install npm@latest -g
    RUN sage -pip install --no-cache-dir --upgrade ipywidgets
    RUN sage -pip install --no-cache-dir dot2tex
    RUN sage -pip install --no-cache-dir RISE
    RUN sage -pip install --no-cache-dir nbdime
    #RUN sage -pip install --no-cache-dir cppyy
    RUN sage -pip install --no-cache-dir git+https://github.com/nthiery/sage-gap-semantic-interface/
    RUN sage -pip install --no-cache-dir git+https://github.com/nthiery/sage-semigroups/
    RUN sage -pip install --no-cache-dir git+https://github.com/zerline/francy-widget/
    RUN sage -pip install --no-cache-dir git+https://github.com/sagemath/sage-combinat-widgets/@develop
    #RUN sage -pip install --no-cache-dir git+https://github.com/sagemath/sage-combinat-widgets/@master
    RUN sage -pip install --no-cache-dir git+https://github.com/sagemath/sage-explorer/@develop

### What it took to get there; lessons learned the hard way

#### Example: Sage interface to GAP

- **Once upon a time:** no interface from my favorite system (MuPAD)
- **2008:** Sage, with it's text interface to GAP!<br>
  Caveats:
  - too slow for low-granularity computation
  - monolithic adapters: only groups can be used as native groups
- **2012:** Sage's libgap: C-level interface to calling GAP! (Volker Braun et al.)<br>
  Caveat: a patched version of GAP
  - hard to maintain
  - hard to package
  - binary incompatible: $\Longrightarrow$ no kernel modules $\Longrightarrow$ no Semigroups
- **2017-2019:** GAP's libgap (Markus Pfeiffer, Sebastian Gutsche, Max Horn, ...)<br>
  Integration in Sage (Erik Bray, ...)<br>
- **2018** OSCAR: fast bidirectional calls between GAP and Julia (Sebastian Gutsche et al.)
- **2015-:** Semantic interface to GAP (T. et al.)<br>
  Prototype; needs maturation and users to expand its scope

#### Example: Interface to Semigroupe / libsemigroups
- **Once upon a time**: Semigroupe by Jean-Éric Pin
- **2010**: Cython bindings (T. et al.)<br>
  Caveats:
  - Semigroupe designed as standalone system. Global variable and single semigroup<br>
  $\Longrightarrow$ Never made its way into Sage
- **2016-**: libsemigroups ( et al.)
- **2017**: Cython bindings (James Mitchell et T.)<br>
  Caveat: too much work to maintain
- **2018**: cppyy bindings<br>
  Caveat: cppyy not yet mature enough
- **2017-2020**: libsemigroups maturation (James Mitchel with help from Florent Hivert, T.):<br>
  API, build system, packaging, ...
- **2019**: libsemigroups+Semigroups packaged in Sage (Dima Pasechnik)
- **2020**: libsemigroups directly usable in Sage via cppyy!

- **Challenge**: how to push features down the stack?

## Take home messages

###### Main challenge of computational semigroup representation theory?
Combining many different building blocks

#####  Major evolution of the last decade: we are building an ecosystem of interoperable software
- A place where software pieces can emerge, live, compete, and die when they are superseded or not used anymore, without drowning down years of work
- A place that fosters new ideas, and healthy competition at the granularity of ideas only

**Hard** but **Doable**

**What it takes**
- Large scale collaboration, across communities
- Elaborating and following best practices; exploit emerging technologies
- Research Software Engineers; funding? people? career paths?

##### Opportunity: European funding
- **Takes:** lots of work and luck; **Gives:** lot of resources
- EOSC, Cost Networks

## Appendices

### Best practices

#### (Re)Designing our systems to be good citizens of the ecosystem
- Free Software
- Tests and continuous integration
- Mainstream build and installation system
- Mainstream development workflow
- Packaging (conda, linux packages, ...)
- Reduced coupling with dependencies
- Well defined low level API to interact with the system
- Modularity
- Exposing the semantic

#### Interfaces

- Language-level interface:
  - Fast comprehensive access to the other system via remote function calls and handles
- Aligning the semantic:
  - Automatic conversion of objects
  - Adapting

#### Externalize and share solutions to common needs
- Reusable specialized high performance libraries
- Development tools
- User interface / interactive environment (Jupyter)
- Interactive widgets (Francy, ...)

Shameless plug:
- Session: **The Jupyter environment for interactive computational mathematics**?<br>
  International Congress for Mathematical Software<br>
  Braunschweig, 13-16 July, 2020<br>
  Call for speakers!

#### Research Software Engineers
- We need software for our research
- It's our job as researcher to implement the maths we need
- We are killing ourselves with the maintenance and technical work

- Can we do something about it?

- Lesson learned from OpenDreamKit:<br>
  A few **Research Software Engineers** can make a major impact

- We need to do something to secure funding and career paths

#### Large scale collaboration
- Constantly coordinating efforts
- Regular joint workshops
- Joint projects?

- Cost network?:  European Cooperation in Science and Technology